<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/BiLSTM-CRF%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bi-LSTM-CRF Model

In [1]:
from google.colab import files
uploaded = files.upload()

Saving prepro.py to prepro (2).py


In [2]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-myfdsxpg
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-myfdsxpg
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=0973858a4a90659dca2a0e9975c64ce1e776b2b9c02a58822ebb346bbea8e34a
  Stored in directory: /tmp/pip-ephem-wheel-cache-v1hnefpr/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [3]:
!pip install sklearn_crfsuite

In [4]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

Using TensorFlow backend.


In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [6]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

words= [w.lower() for w in words]

setlabel=list(set(tags))
setlabel.remove('O')
        
words=list(set(words))
n_words = len(words)
print("Number of words in the dataset: ", n_words)
n_tags = len(list(set(tags)))
print("Number of Labels: ", n_tags)

Number of words in the dataset:  24277
Number of Labels:  5


In [7]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(list(set(tags)))}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["ORG"]))

The labels LOC (location) is identified by the index: 5


In [8]:
word2idx = {w: i + 2 for i, w in enumerate(list(set(words)))}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 16931


In [13]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [14]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0].lower()] for w in s] for s in train]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X[0]

array([ 2614, 16931, 10422, 18946, 12450,  9321, 14180,  4607, 15674,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)

In [15]:
def get_labeler(word):
  if word =="O":
    return word
  elif "MISC" in word:
    return word[-4:]
  else:
    return word[-3:]

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y[0]

array([5, 4, 1, 4, 4, 4, 1, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

## Building LSTM-CRF Model

In [0]:
# Model definition
EMBEDDING=50

input = Input(shape=(MAX_LEN,))

emb = Embedding(input_dim=len(word2idx)+1, output_dim=EMBEDDING, # n_words + 2 (PAD & UNK)
                  input_length=MAX_LEN, mask_zero=True)(input)

lstm1 = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(emb)


lstm2 = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(lstm1)

model = TimeDistributed(Dense(10, activation="relu"))(lstm2)
crf = CRF(n_tags+1)  # CRF layer - 6 tags
out = crf(model)  # output
model = Model(input, out)

In [26]:
model.compile(optimizer="adam", loss=crf_loss,metrics=[crf_viterbi_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 124)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 124, 50)           1214000   
_________________________________________________________________
bidirectional_9 (Bidirection (None, 124, 128)          58880     
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 124, 128)          98816     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 124, 10)           1290      
_________________________________________________________________
crf_5 (CRF)                  (None, 124, 6)            114       
Total params: 1,373,100
Trainable params: 1,373,100
Non-trainable params: 0
_________________________________________________________________


In [27]:
%%time
BATCH_SIZE = 200
EPOCHS=16

history = model.fit(X, np.array(y), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1)

W0804 21:55:21.829792 140128092747648 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 11232 samples, validate on 2809 samples
Epoch 1/16
11232/11232 [==============================] - 83s 7ms/step - loss: 33.8672 - crf_viterbi_accuracy: 0.6700 - val_loss: 29.4873 - val_crf_viterbi_accuracy: 0.8462
Epoch 2/16
11232/11232 [==============================] - 76s 7ms/step - loss: 33.3770 - crf_viterbi_accuracy: 0.8243 - val_loss: 29.1478 - val_crf_viterbi_accuracy: 0.8587
Epoch 3/16
11232/11232 [==============================] - 76s 7ms/step - loss: 33.0249 - crf_viterbi_accuracy: 0.8703 - val_loss: 28.9892 - val_crf_viterbi_accuracy: 0.8970
Epoch 4/16
11232/11232 [==============================] - 76s 7ms/step - loss: 32.8081 - crf_viterbi_accuracy: 0.9319 - val_loss: 28.9578 - val_crf_viterbi_accuracy: 0.9124
Epoch 5/16
11232/11232 [==============================] - 76s 7ms/step - loss: 32.7443 - crf_viterbi_accuracy: 0.9510 - val_loss: 28.9598 - val_crf_viterbi_accuracy: 0.9121
Epoch 6/16
11232/11232 [==============================] - 75s 7ms/step - loss: 32.7110

## Predict on Training Set

In [28]:
pred = model.predict(X, verbose=1)

14041/14041 [==============================] - 198s 14ms/step


In [29]:
# TRain Eval
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

              precision    recall  f1-score   support

         LOC       0.96      0.93      0.94      8297
        MISC       0.85      0.91      0.88      4593
           O       0.99      0.98      0.99    169578
         ORG       0.90      0.90      0.90     10025
         PAD       1.00      1.00      1.00   1537463
         PER       0.88      0.98      0.92     11128

    accuracy                           1.00   1741084
   macro avg       0.93      0.95      0.94   1741084
weighted avg       1.00      1.00      1.00   1741084

0.9973119557623062


In [30]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

        MISC       0.85      0.91      0.88      4593
         PER       0.88      0.98      0.92     11128
         LOC       0.96      0.93      0.94      8297
         ORG       0.90      0.90      0.90     10025

   micro avg       0.90      0.94      0.92     34043
   macro avg       0.89      0.93      0.91     34043
weighted avg       0.90      0.94      0.92     34043

0.9151510527416556


## Predict on Test Set

In [0]:

# Convert each sentence from list of Token to list of word_index
X_test = [[word2idx[w[0].lower()] for w in s] for s in test]

# Padding each sentence to have the same lenght
X_test = pad_sequences(maxlen=MAX_LEN, sequences=X_test, padding="post", value=word2idx["PAD"])
X_test

# Convert Tag/Label to tag_index
y_test = [[tag2idx[get_labeler((w[1]))] for w in s] for s in test]

# Padding each sentence to have the same lenght
y_test = pad_sequences(maxlen=MAX_LEN, sequences=y_test, padding="post", value=tag2idx["PAD"])

# One-Hot encode categorical labels
y_test = [to_categorical(i, num_classes=n_tags+1) for i in y_test]

In [33]:
pred = model.predict(X_test, verbose=1)

3453/3453 [==============================] - 50s 15ms/step


In [35]:
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y_test, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

        MISC       0.51      0.65      0.57       918
         PER       0.57      0.89      0.70      2773
         LOC       0.76      0.79      0.78      1925
         ORG       0.69      0.59      0.64      2496

   micro avg       0.63      0.75      0.68      8112
   macro avg       0.63      0.73      0.67      8112
weighted avg       0.65      0.75      0.68      8112

0.6833202768124159
